In [1]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
# print(x.shape)

# print(x[0].shape)
print(x[1].shape)
print(x[0, 0])

(1, 28, 28)
[[0.45096698 0.03233247 0.60855993 0.77091087 0.30640471 0.33485734
  0.57485591 0.80482116 0.46690008 0.41608242 0.90396661 0.21019314
  0.17103223 0.56973328 0.50356967 0.8723645  0.49833594 0.08739513
  0.33398042 0.76397843 0.8595235  0.36430621 0.0591059  0.47317479
  0.66889136 0.99316777 0.88087527 0.03504925]
 [0.25901447 0.11584528 0.18597811 0.08691921 0.61486277 0.25145265
  0.40687835 0.32995766 0.72311984 0.70662932 0.27678654 0.20566532
  0.09485382 0.69231077 0.67584314 0.95222185 0.5641843  0.9657282
  0.97354343 0.49497667 0.63661207 0.61556955 0.88831664 0.48127079
  0.30170561 0.91762775 0.34092068 0.9505299 ]
 [0.10454763 0.77940396 0.49532002 0.02874688 0.28945412 0.8873185
  0.78217551 0.53846664 0.95477951 0.30391213 0.49703151 0.23135452
  0.22367298 0.43905235 0.2062253  0.72493544 0.13525088 0.22154006
  0.89872544 0.16358567 0.84953992 0.30162521 0.18332438 0.97864003
  0.39649185 0.31658376 0.61150472 0.03879641]
 [0.87915725 0.71826285 0.1852512

## numpy에서 for 문을 사용하면 성능이 떨어짐!

## for문 대신에 im2col로 데이터 전개하기

In [ ]:
# im2col 함수에서 행렬곱을 하여 convolution을 바로 구현하지 않고
# im2col 과 transpose된 벡터(커널)를 행렬곱 하는 이유는 backpropagation을 효율적으로 구현하기 위해!! 행렬곱은 미분이 되므로!!
# 강사님은 for loop는 미분이 없다라고 하셨는데 for loop는 미분하기에 효율적이지 않음!!
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N, out_h, out_w, C, filter_h, filter_w)
    return col

In [22]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


# np.pad 확인

In [ ]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
print(input_data)
pad = 1
np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

[[[[ 1  2]
   [ 3  4]]

  [[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]

  [[13 14]
   [15 16]]]]


array([[[[ 0,  0,  0,  0],
         [ 0,  1,  2,  0],
         [ 0,  3,  4,  0],
         [ 0,  0,  0,  0]],

        [[ 0,  0,  0,  0],
         [ 0,  5,  6,  0],
         [ 0,  7,  8,  0],
         [ 0,  0,  0,  0]]],


       [[[ 0,  0,  0,  0],
         [ 0,  9, 10,  0],
         [ 0, 11, 12,  0],
         [ 0,  0,  0,  0]],

        [[ 0,  0,  0,  0],
         [ 0, 13, 14,  0],
         [ 0, 15, 16,  0],
         [ 0,  0,  0,  0]]]])

In [2]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
print(input_data)
pad = 1
np.pad(input_data, [(0,0), (0,0), (0, 0), (0, pad)], 'constant')

[[[[ 1  2]
   [ 3  4]]

  [[ 5  6]
   [ 7  8]]]


 [[[ 9 10]
   [11 12]]

  [[13 14]
   [15 16]]]]


array([[[[ 1,  2,  0],
         [ 3,  4,  0]],

        [[ 5,  6,  0],
         [ 7,  8,  0]]],


       [[[ 9, 10,  0],
         [11, 12,  0]],

        [[13, 14,  0],
         [15, 16,  0]]]])

In [17]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
pad = 1
np.pad(input_data, [(pad, 0), (0, 0), (0,0), (0,0)], 'constant')

array([[[[ 0,  0],
         [ 0,  0]],

        [[ 0,  0],
         [ 0,  0]]],


       [[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]]]])

In [35]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])
pad = 1
np.pad(input_data, [(0, 0), (0, pad), (0,0), (0,0)], 'constant')

array([[[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]],

        [[ 0,  0],
         [ 0,  0]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]],

        [[ 0,  0],
         [ 0,  0]]]])

# im2col 작동 원리 디버깅

In [ ]:
img = np.arange(1, 1*3*4*4 + 1).reshape(1, 3, 4, 4)
img

array([[[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12],
         [13, 14, 15, 16]],

        [[17, 18, 19, 20],
         [21, 22, 23, 24],
         [25, 26, 27, 28],
         [29, 30, 31, 32]],

        [[33, 34, 35, 36],
         [37, 38, 39, 40],
         [41, 42, 43, 44],
         [45, 46, 47, 48]]]])

In [13]:
img = np.arange(1, 1*3*4*4 + 1).reshape(1, 3, 4, 4)
stride = 1
pad = 0
filter_h = filter_w = 3
N, C, H, W = img.shape
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

y, x = 0, 2
y_max = y + stride * 2
x_max = x + stride * 2

col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride] # 2 번째 축 0으로 고정, 3 번째 축 2로 고정 
col.shape, col

((1, 3, 3, 3, 2, 2),
 array([[[[[[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 3.,  4.],
            [ 7.,  8.]]],
 
 
          [[[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]]],
 
 
          [[[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]]]],
 
 
 
         [[[[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]],
 
           [[19., 20.],
            [23., 24.]]],
 
 
          [[[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]]],
 
 
          [[[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]],
 
           [[ 0.,  0.],
            [ 0.,  0.]]]],
 
 
 
         [[[[ 0.,  0.],
            [

In [ ]:
r_col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N*out_h*out_w, C*filter_h*filter_w)
r_col.shape, r_col # (1*2*2, 3*3*3) = (4, 27)

((4, 27),
 array([[ 0.,  0.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 19.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0., 35.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 20.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0., 36.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 23.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0., 39.,  0.,  0.,  0.,  0.,  0.,
          0.],
        [ 0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 24.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0., 40.,  0.,  0.,  0.,  0.,  0.,
          0.]]))

In [38]:
input_data = np.array([[[[1, 2], 
                         [3, 4]],
                        [[5, 6],
                         [7, 8]]],
                       [[[9, 10],
                         [11, 12]],
                        [[13, 14],
                         [15, 16]]]])

pad = 0
N, C, filter_h, filter_w, out_h, out_w = 2, 2, 1, 1, 2, 2
stride=1
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

In [27]:
img

array([[[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]]]])

In [28]:
col

array([[[[[[ 1.,  2.],
           [ 3.,  4.]]]],



        [[[[ 5.,  6.],
           [ 7.,  8.]]]]],




       [[[[[ 9., 10.],
           [11., 12.]]]],



        [[[[13., 14.],
           [15., 16.]]]]]])

In [ ]:
col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N*out_h*out_w, C*filter_h*filter_w)
col

array([[ 1.,  5.],
       [ 2.,  6.],
       [ 3.,  7.],
       [ 4.,  8.],
       [ 9., 13.],
       [10., 14.],
       [11., 15.],
       [12., 16.]])

In [40]:
input_data = np.arange(1, 4*3*4*4 + 1).reshape(4, 3, 4, 4)

pad = 0
N, C, filter_h, filter_w, out_h, out_w = 4, 3, 3, 3, 2, 2
stride=1
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') # N, C: 패딩없음 (0, 0)
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
        print(col)
        break

[[[[[[  1.   2.]
     [  5.   6.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]


   [[[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]


   [[[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]]



  [[[[ 17.  18.]
     [ 21.  22.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]


   [[[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]


   [[[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]]



  [[[[ 33.  34.]
     [ 37.  38.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]


   [[[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]


   [[[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]

    [[  0.   0.]
     [  0.   0.]]]]]




 [

In [31]:
img

array([[[[  1,   2,   3,   4],
         [  5,   6,   7,   8],
         [  9,  10,  11,  12],
         [ 13,  14,  15,  16]],

        [[ 17,  18,  19,  20],
         [ 21,  22,  23,  24],
         [ 25,  26,  27,  28],
         [ 29,  30,  31,  32]],

        [[ 33,  34,  35,  36],
         [ 37,  38,  39,  40],
         [ 41,  42,  43,  44],
         [ 45,  46,  47,  48]]],


       [[[ 49,  50,  51,  52],
         [ 53,  54,  55,  56],
         [ 57,  58,  59,  60],
         [ 61,  62,  63,  64]],

        [[ 65,  66,  67,  68],
         [ 69,  70,  71,  72],
         [ 73,  74,  75,  76],
         [ 77,  78,  79,  80]],

        [[ 81,  82,  83,  84],
         [ 85,  86,  87,  88],
         [ 89,  90,  91,  92],
         [ 93,  94,  95,  96]]],


       [[[ 97,  98,  99, 100],
         [101, 102, 103, 104],
         [105, 106, 107, 108],
         [109, 110, 111, 112]],

        [[113, 114, 115, 116],
         [117, 118, 119, 120],
         [121, 122, 123, 124],
         [125, 126, 1

In [32]:
col

array([[[[[[  1.,   2.],
           [  5.,   6.]],

          [[  2.,   3.],
           [  6.,   7.]],

          [[  3.,   4.],
           [  7.,   8.]]],


         [[[  5.,   6.],
           [  9.,  10.]],

          [[  6.,   7.],
           [ 10.,  11.]],

          [[  7.,   8.],
           [ 11.,  12.]]],


         [[[  9.,  10.],
           [ 13.,  14.]],

          [[ 10.,  11.],
           [ 14.,  15.]],

          [[ 11.,  12.],
           [ 15.,  16.]]]],



        [[[[ 17.,  18.],
           [ 21.,  22.]],

          [[ 18.,  19.],
           [ 22.,  23.]],

          [[ 19.,  20.],
           [ 23.,  24.]]],


         [[[ 21.,  22.],
           [ 25.,  26.]],

          [[ 22.,  23.],
           [ 26.,  27.]],

          [[ 23.,  24.],
           [ 27.,  28.]]],


         [[[ 25.,  26.],
           [ 29.,  30.]],

          [[ 26.,  27.],
           [ 30.,  31.]],

          [[ 27.,  28.],
           [ 31.,  32.]]]],



        [[[[ 33.,  34.],
           [ 37.,  38.]

In [ ]:
col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1) # (N*out_h*out_w, C*filter_h*filter_w)
col

array([[  1.,   2.,   3.,   5.,   6.,   7.,   9.,  10.,  11.,  17.,  18.,
         19.,  21.,  22.,  23.,  25.,  26.,  27.,  33.,  34.,  35.,  37.,
         38.,  39.,  41.,  42.,  43.],
       [  2.,   3.,   4.,   6.,   7.,   8.,  10.,  11.,  12.,  18.,  19.,
         20.,  22.,  23.,  24.,  26.,  27.,  28.,  34.,  35.,  36.,  38.,
         39.,  40.,  42.,  43.,  44.],
       [  5.,   6.,   7.,   9.,  10.,  11.,  13.,  14.,  15.,  21.,  22.,
         23.,  25.,  26.,  27.,  29.,  30.,  31.,  37.,  38.,  39.,  41.,
         42.,  43.,  45.,  46.,  47.],
       [  6.,   7.,   8.,  10.,  11.,  12.,  14.,  15.,  16.,  22.,  23.,
         24.,  26.,  27.,  28.,  30.,  31.,  32.,  38.,  39.,  40.,  42.,
         43.,  44.,  46.,  47.,  48.],
       [ 49.,  50.,  51.,  53.,  54.,  55.,  57.,  58.,  59.,  65.,  66.,
         67.,  69.,  70.,  71.,  73.,  74.,  75.,  81.,  82.,  83.,  85.,
         86.,  87.,  89.,  90.,  91.],
       [ 50.,  51.,  52.,  54.,  55.,  56.,  58.,  59.,  60.,  66

In [ ]:
col.shape # (4*2*2, 3*3*3)

(16, 27)